In [1]:
import time
from collections import deque
import numpy as np
import serial


In [7]:
import time
from collections import deque
import numpy as np
import serial

# ---------------- CONFIG ----------------
PORT = "COM20"
BAUD = 115200

EXPECTED_COLS = 14  # SEQ,T_US,...,G2Z

# Same values you used offline (tune if needed)
SMOOTH_WIN = 7
START_THR = 0.22
STOP_THR  = 0.09

MIN_REST_S = 0.15       # stop confirm
MIN_REP_DUR_S = 0.35    # ignore tiny reps at end

# NEW (but still same FSM idea, just symmetric debounce)
START_CONFIRM_S = 0.08  # must stay above START_THR this long to start (e.g., 80 ms)
COOLDOWN_S = 0.10       # after end, ignore starts briefly (prevents immediate retrigger)

DEBUG_EVENTS = True     # print trigger values at START/END
# ---------------------------------------

def compute_dw_mag_sample(g1x, g1y, g1z, g2x, g2y, g2z) -> float:
    w1 = np.array([g1x, g1y, g1z], dtype=np.float64)
    w2 = np.array([g2x, g2y, g2z], dtype=np.float64)
    return float(np.linalg.norm(w1 - w2))

class CausalMovingAverage:
    def __init__(self, win: int):
        self.win = max(1, int(win))
        self.buf = deque(maxlen=self.win)
        self.sum = 0.0

    def update(self, x: float) -> float:
        if self.win == 1:
            return x
        if len(self.buf) == self.buf.maxlen:
            self.sum -= self.buf[0]
        self.buf.append(x)
        self.sum += x
        return self.sum / len(self.buf)

class RepStateMachine:
    IDLE = 0
    IN_REP = 1

    def __init__(self):
        self.state = self.IDLE

        self.rep_count = 0
        self.rep_start_t = None

        # confirmation timers
        self.above_start_time = 0.0
        self.below_stop_time = 0.0

        # cooldown
        self.cooldown_until = -1e9

        # ROM accumulator
        self.rom_rad = 0.0

        # last sample
        self.last_t = None
        self.last_dw_raw = None

    def update(self, t_s: float, dw_raw: float, dw_s: float):
        if self.last_t is None:
            self.last_t = t_s
            self.last_dw_raw = dw_raw
            return

        dt = t_s - self.last_t
        # guard timestamp glitches
        if dt <= 0 or dt > 0.2:
            self.last_t = t_s
            self.last_dw_raw = dw_raw
            return

        # ---------------- IDLE ----------------
        if self.state == self.IDLE:
            # cooldown prevents immediate retrigger
            if t_s < self.cooldown_until:
                self.above_start_time = 0.0
            else:
                # START confirmation (debounce)
                if dw_s >= START_THR:
                    self.above_start_time += dt
                else:
                    self.above_start_time = 0.0

                if self.above_start_time >= START_CONFIRM_S:
                    self.state = self.IN_REP
                    self.rep_start_t = t_s
                    self.rep_count += 1

                    # reset rep accumulators
                    self.rom_rad = 0.0
                    self.below_stop_time = 0.0

                    if DEBUG_EVENTS:
                        print(f"[{t_s:8.3f}s] REP START (rep={self.rep_count}) "
                              f"dw_s={dw_s:.3f} thr={START_THR:.2f} "
                              f"confirm={self.above_start_time:.3f}s dw_raw={dw_raw:.3f}")

        # ---------------- IN_REP ----------------
        elif self.state == self.IN_REP:
            # accumulate ROM: trapezoid ∫|Δω| dt (same math as offline trapz, but streaming)
            self.rom_rad += 0.5 * (self.last_dw_raw + dw_raw) * dt

            # STOP confirmation
            if dw_s <= STOP_THR:
                self.below_stop_time += dt
            else:
                self.below_stop_time = 0.0

            if self.below_stop_time >= MIN_REST_S:
                rep_dur = t_s - (self.rep_start_t if self.rep_start_t is not None else t_s)
                rom_deg = self.rom_rad * (180.0 / np.pi)

                if rep_dur >= MIN_REP_DUR_S:
                    if DEBUG_EVENTS:
                        print(f"[{t_s:8.3f}s] REP END   (rep={self.rep_count}, dur={rep_dur:.2f}s) "
                              f"dw_s={dw_s:.3f} thr={STOP_THR:.2f} rest={self.below_stop_time:.3f}s")
                    print(f"            ROM_PROXY = {rom_deg:.2f} deg")
                else:
                    if DEBUG_EVENTS:
                        print(f"[{t_s:8.3f}s] REP END (IGNORED short rep: {rep_dur:.2f}s, ROM={rom_deg:.2f})")

                # go idle + cooldown
                self.state = self.IDLE
                self.rep_start_t = None
                self.above_start_time = 0.0
                self.cooldown_until = t_s + COOLDOWN_S
                self.rom_rad = 0.0
                self.below_stop_time = 0.0

        # update last sample memory
        self.last_t = t_s
        self.last_dw_raw = dw_raw

def parse_line(line: str):
    parts = line.strip().split(",")
    if len(parts) != EXPECTED_COLS:
        return None
    try:
        t_us = int(float(parts[1]))
        t_s = t_us / 1e6

        g1x = float(parts[5]);  g1y = float(parts[6]);  g1z = float(parts[7])
        g2x = float(parts[11]); g2y = float(parts[12]); g2z = float(parts[13])

        return t_s, g1x, g1y, g1z, g2x, g2y, g2z
    except ValueError:
        return None

def main():
    ser = serial.Serial(PORT, BAUD, timeout=1)
    print(f"Connected to {PORT} @ {BAUD}")

    time.sleep(2.0)
    ser.reset_input_buffer()

    smoother = CausalMovingAverage(SMOOTH_WIN)
    fsm = RepStateMachine()

    while True:
        raw = ser.readline()
        if not raw:
            continue
        line = raw.decode(errors="ignore").strip()
        if not line:
            continue
        if line.startswith("SEQ") or "A1X" in line:
            continue

        parsed = parse_line(line)
        if parsed is None:
            continue

        t_s, g1x, g1y, g1z, g2x, g2y, g2z = parsed

        dw_raw = compute_dw_mag_sample(g1x, g1y, g1z, g2x, g2y, g2z)
        dw_s = smoother.update(dw_raw)

        fsm.update(t_s, dw_raw, dw_s)

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\nStopped.")


Connected to COM20 @ 115200
[ 764.182s] REP START (rep=1) dw_s=0.239 thr=0.22 confirm=0.080s dw_raw=0.347
[ 764.922s] REP END   (rep=1, dur=0.74s) dw_s=0.065 thr=0.20 rest=0.160s
            ROM_PROXY = 11.97 deg
[ 767.302s] REP START (rep=2) dw_s=0.242 thr=0.22 confirm=0.090s dw_raw=0.101
[ 767.652s] REP END   (rep=2, dur=0.35s) dw_s=0.165 thr=0.20 rest=0.160s
            ROM_PROXY = 3.07 deg
[ 770.152s] REP START (rep=3) dw_s=0.515 thr=0.22 confirm=0.080s dw_raw=0.280
[ 770.452s] REP END (IGNORED short rep: 0.30s, ROM=3.72)
[ 771.042s] REP START (rep=4) dw_s=0.777 thr=0.22 confirm=0.090s dw_raw=0.945
[ 772.452s] REP END   (rep=4, dur=1.41s) dw_s=0.112 thr=0.20 rest=0.160s
            ROM_PROXY = 41.60 deg

Stopped.
